# Restaurant near you

In [ ]:
import py2neo
from py2neo import Graph, Node, Relationship
import requests
import json

#open Graph database
graph=Graph("localhost")  #depend on your authorization choice


#send url to get your current location #can use better method to get the division automatically as well
send_url = 'http://freegeoip.net/json'
r = requests.get(send_url)
j = json.loads(r.text)
lati = j['latitude']
longi = j['longitude']

#query for the match your location with the restaurant
query='''
WITH point({latitude:{lat},longitude:{lon}}) AS mapcenter
   MATCH (a:Restaurant)-[:IN_DIVISION]->(b:Division{e_name:{division}}) 
   WITH a, distance (point({latitude: a.latitude, longitude: a.longitude}), mapcenter) AS distance 
     //near you but we can just do the limit so that only shows
   RETURN a.shopId, distance
   ORDER BY distance LIMIT 1000
'''
lat=24.8229533748
lon=121.771853579
division="Jiaoxi Township"
graph.run(query,lat=lati,lon=longi,division=division).data()

# restaurant by month, most review with SDRate greater than 3

In [2]:
import py2neo
from py2neo import Graph, Node, Relationship

#open Graph database
graph=Graph("localhost")  #depend on your authorization choice


#recommendation for restaurant by month: ranking the the restaurant by their review rating: most high reviews of the month
query='''
match (d:Division{e_name:{division}})-[:IN_MONTH]->(m:Month{month:{month}})
match	(m)<-[:WRITTEN_IN]-(r:Review)
match   (r)-[:ABOUT]->(n:Restaurant) where r.SDRate>3
return n.shopId, count(r) as count order by count desc
'''
month=1
division="Jiaoxi Township"
graph.run(query,month=month,division=division).data()

[{'count': 4, 'n.shopId': 86266},
 {'count': 2, 'n.shopId': 1017400},
 {'count': 1, 'n.shopId': 38366},
 {'count': 1, 'n.shopId': 35898},
 {'count': 1, 'n.shopId': 37661},
 {'count': 1, 'n.shopId': 956260},
 {'count': 1, 'n.shopId': 54127}]

# Restaurant by month, most review with SDRate> average

In [17]:
import py2neo
from py2neo import Graph, Node, Relationship

#open Graph database
graph=Graph("localhost")  #depend on your authorization choice

#input month, division, average
month=1
division="Jiaoxi Township"

query='''
match (d:Division{e_name:"Jiaoxi Township"})-[:IN_MONTH]->(m:Month{month:5}),
	(m)<-[:WRITTEN_IN]-(r:Review),
    (r)-[:ABOUT]->(n:Restaurant)
return avg(r.SDRate) as avg
'''
avg_data=graph.run(query,month=month,division=division).data()

average=avg_data[0]['avg']

#recommendation for restaurant by month: ranking the the restaurant by their review rating: most review above average of month
query='''
match (d:Division{e_name:{division}})-[:IN_MONTH]->(m:Month{month:{month}})
match	(m)<-[:WRITTEN_IN]-(r:Review)
match   (r)-[:ABOUT]->(n:Restaurant) where r.SDRate>{average}
return n.shopId, count(r) as count order by count desc
'''
graph.run(query,month=month,division=division,average=average).data()

[{'count': 4, 'n.shopId': 86266},
 {'count': 2, 'n.shopId': 1017400},
 {'count': 1, 'n.shopId': 38366},
 {'count': 1, 'n.shopId': 35898},
 {'count': 1, 'n.shopId': 37661},
 {'count': 1, 'n.shopId': 956260},
 {'count': 1, 'n.shopId': 54127}]